# Modeling & Fitting

In [1]:
import os
import sys
sys.path.insert(0, "../../")

import mpld3
mpld3.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

from core import util, config, database
from core.data.model import ModelDataRepo
from core.data.model.entities import *

database.init()
config.init_plot()

### Get Data

In [2]:
selected_kabko = "KOTA SURABAYA"
kabko = ModelDataRepo.get_kabko_full(selected_kabko)

### Make Model

In [3]:
from core.modeling import SeicrdRlcModel

mod = SeicrdRlcModel(kabko)

### Select Datsets to Use for Fitting

In [4]:
used_datasets = [
    #"infectious", 
    "critical_cared", 
    "infectious_all", 
    "recovered", 
    "dead",
    #"infected"
]
mod.use_datasets(used_datasets)

### List Variables that Can be Fitted

In [5]:
varying_init = [
    "death_time_over",
    "r_over", 
    "kapasitas_rs_mul", 
    "test_coverage_max", 
    "test_coverage_0", "test_coverage_increase", 
    "incubation_period", 
    "critical_time", "critical_chance",
    "death_time_normal", "death_chance_normal", 
    "k", "r_0", "r_1", "r_2"
]

### Select Variables to Fit

In [6]:
varying_now = [
    #"death_time_over",
    #"r_over", 
    #"kapasitas_rs_mul", 
    #"test_coverage_max", 
    #"test_coverage_0", "test_coverage_increase", 
    "incubation_period", 
    "critical_time", "critical_chance",
    "death_time_normal", "death_chance_normal", 
    "k", "r_0", "r_1", "r_2"
]
unvary = [x for x in varying_init if x not in varying_now]

### Fit!

In [ ]:
result = mod.fit(test_splits=[2], unvary=unvary)
result

### Fit Results

Params Should Have "standard error" and "relative error" Columns. If they dont:
1. A parameter might not be significant
2. A parameter has hit its min/max limit and can't vary

In [ ]:
result.fit_result.params

### Model Scores

In [ ]:
result.test_scorer.mean_absolute_error()

In [ ]:
result.test_scorer.explained_variance_score()

In [ ]:
result.fit_scorer.mean_absolute_error()

In [ ]:
result.fit_scorer.explained_variance_score()

### Predict

In [ ]:
values = dict(result.fit_result.values)
values["days"] += 30
model_result = mod.model(**values)

### Plot!

In [ ]:
from core.modeling import ModelPlotter

plotter = ModelPlotter(model_result)

In [ ]:
length = kabko.data_count + result.outbreak_shift
length

In [ ]:
import matplotlib.pyplot as plt
for d in used_datasets:
    datasets = kabko.get_datasets([d], result.outbreak_shift)
    plotter.plot(
        plotter.plot_main_data, 
        datasets,
        length
    )
    plt.show()

In [ ]:
plotter.plot(plotter.plot_main_lite);

In [ ]:
plotter.plot(plotter.plot_daily_lite);

In [ ]:
plotter.plot(plotter.plot_r0);

In [ ]:
plotter.plot(plotter.plot_mortality_rate);

In [ ]:
plotter.plot(plotter.plot_over);

In [ ]:
plotter.plot(plotter.plot_healthcare);

### Save Params

Run code below ONLY if you want to save the parameters.

In [ ]:
ModelDataRepo.update_all(result.kabko, result.fit_result.values, result.outbreak_shift)

### Confidence Interval

This requires standard error to be present for each params.

In [ ]:
result.fit_result.conf_interval()